In [2]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_molpcba/'
dataset = GraphPropPredDataset(name = 'ogbg-molpcba')


split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [3]:
print(dataset[0][0].keys())

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])


In [4]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [1]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    labels.append(label)
    num_nodes += g["num_nodes"]
print(len(edges))
print(num_nodes)

NameError: name 'tqdm' is not defined

In [6]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [7]:
_node_list = sparse.lil_matrix((len(dataset), num_nodes))
p=0
for i, indices in enumerate(node_list):
    p=p+1
    _node_list[i, indices] = 1
print(p)

437929


In [10]:
data = {
    "node_feats": _node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
np.savez_compressed("ogbg-molpcba.npz", **data)

In [9]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
np.savez_compressed("ogbg-molpcba_task.npz",**task_data)